In [1]:
import pandas  as pd
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
import numpy as np
import math
from statsmodels.tsa.stattools import acf, pacf
import statsmodels.tsa.stattools as ts
from statsmodels.tsa.arima_model import ARIMA


In [11]:
df= pd.read_excel("typea.xlsx")
price = df["Monthly Consumption of Type A Medicine"][:-2]
lnprice = np.log(price)
plt.plot(price)
plt.show()

In [12]:
acf_1 =  acf(lnprice)[1:20]
plt.plot(acf_1)
plt.show()
test_df = pandas.DataFrame([acf_1]).T
test_df.columns = ['Pandas Autocorrelation']
test_df.index += 1
test_df.plot(kind='bar')
pacf_1 =  pacf(lnprice)[1:20]
plt.plot(pacf_1)
plt.show()
test_df = pandas.DataFrame([pacf_1]).T
test_df.columns = ['Pandas Partial Autocorrelation']
test_df.index += 1
test_df.plot(kind='bar')
result = ts.adfuller(lnprice, 1)
result

G:\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1283: RuntimeWarning: invalid value encountered in sqrt
  return rho, np.sqrt(sigmasq)
G:\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1275: RuntimeWarning: invalid value encountered in double_scalars
  r[k] = (X[0:-k] * X[k:]).sum() / denom(k)


(-2.9865149161338365,
 0.036173072744222996,
 1,
 34,
 {'1%': -3.639224104416853,
  '10%': -2.614446989619377,
  '5%': -2.9512301791166293},
 -37.56004634645299)

In [13]:
lnprice_diff=lnprice-lnprice.shift()
diff=lnprice_diff.dropna()
acf_1_diff =  acf(diff)[1:20]
test_df = pandas.DataFrame([acf_1_diff]).T
test_df.columns = ['First Difference Autocorrelation']
test_df.index += 1
test_df.plot(kind='bar')
pacf_1_diff =  pacf(diff)[1:20]
plt.plot(pacf_1_diff)
plt.show()


G:\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1283: RuntimeWarning: invalid value encountered in sqrt
  return rho, np.sqrt(sigmasq)
G:\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1275: RuntimeWarning: invalid value encountered in double_scalars
  r[k] = (X[0:-k] * X[k:]).sum() / denom(k)


In [35]:
price_matrix=lnprice.as_matrix()
model = ARIMA(price_matrix, order=(0,1,3))
model_fit = model.fit(disp=0)
print(model_fit.summary())

                             ARIMA Model Results                              
Dep. Variable:                    D.y   No. Observations:                   35
Model:                 ARIMA(0, 1, 3)   Log Likelihood                  18.290
Method:                       css-mle   S.D. of innovations              0.143
Date:                Fri, 07 Sep 2018   AIC                            -26.581
Time:                        09:53:56   BIC                            -18.804
Sample:                             1   HQIC                           -23.896
                                                                              
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0076      0.037      0.207      0.837      -0.065       0.080
ma.L1.D.y      0.1127      0.177      0.636      0.530      -0.235       0.460
ma.L2.D.y      0.1681      0.145      1.159      0.2

G:\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if __name__ == '__main__':


In [36]:
predictions=model_fit.predict(36, 39, typ='levels')
predictions
predictionsadjusted=np.exp(predictions)
predictionsadjusted


array([1947.46332036, 1869.53157118, 1803.00430632, 1816.81066561])

In [2]:


df= pd.read_excel("typea.xlsx")
df = df[:-2]
df["date_time"] = pd.Series(np.random.randn(36), index=df.index)
df["date_time"] = df["Year"].map(str) +"-"+ df["Month"] + "-01"
df.index = pd.to_datetime(df["date_time"])
df.index
price = df["Monthly Consumption of Type A Medicine"]
price.index = df.index


from statsmodels.tsa.seasonal import seasonal_decompose
result = seasonal_decompose(price, model='multiplicative')
result
result.plot()
plt.show()

In [4]:
from pyramid.arima import auto_arima
stepwise_model = auto_arima(price, start_p=1, start_q=1,
                           max_p=3, max_q=3, m=12,
                           start_P=0, seasonal=True,
                           d=1, D=1, trace=True,
                           error_action='ignore',  
                           suppress_warnings=True, 
                           stepwise=True)
print(stepwise_model.aic())

Fit ARIMA: order=(1, 1, 1) seasonal_order=(0, 1, 1, 12); AIC=nan, BIC=nan, Fit time=nan seconds
Fit ARIMA: order=(0, 1, 0) seasonal_order=(0, 1, 0, 12); AIC=315.727, BIC=317.998, Fit time=0.016 seconds
Fit ARIMA: order=(1, 1, 0) seasonal_order=(1, 1, 0, 12); AIC=314.599, BIC=319.141, Fit time=0.172 seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(0, 1, 1, 12); AIC=nan, BIC=nan, Fit time=nan seconds
Fit ARIMA: order=(1, 1, 0) seasonal_order=(0, 1, 0, 12); AIC=314.443, BIC=317.849, Fit time=0.078 seconds
Fit ARIMA: order=(1, 1, 0) seasonal_order=(0, 1, 1, 12); AIC=nan, BIC=nan, Fit time=nan seconds
Fit ARIMA: order=(1, 1, 0) seasonal_order=(1, 1, 1, 12); AIC=nan, BIC=nan, Fit time=nan seconds
Fit ARIMA: order=(2, 1, 0) seasonal_order=(0, 1, 0, 12); AIC=310.524, BIC=315.066, Fit time=0.162 seconds
Fit ARIMA: order=(2, 1, 1) seasonal_order=(0, 1, 0, 12); AIC=302.703, BIC=308.380, Fit time=0.298 seconds
Fit ARIMA: order=(3, 1, 2) seasonal_order=(0, 1, 0, 12); AIC=303.631, BIC=311.579, Fit

In [6]:
stepwise_model.fit(price)

ARIMA(callback=None, disp=0, maxiter=50, method=None, order=(2, 1, 2),
   out_of_sample_size=0, scoring='mse', scoring_args={},
   seasonal_order=(1, 1, 0, 12), solver='lbfgs', start_params=None,
   suppress_warnings=True, transparams=True, trend='c')

In [7]:
future_forecast = stepwise_model.predict(n_periods=5)
future_forecast

array([1946.80878656, 2583.99187331, 2653.79357918, 2944.77091035,
       3432.53699495])

In [3]:
from pyramid.arima import auto_arima